# Tsunami wave propagation

<img src="/img_oilspill.png" width=300 height=200/>

## Information
This problem is about a simulated tsunami wave propagating in the ocean. Such a problem is well-approximated by the shallow water wave equation:

$\frac{\partial^{2}\eta}{\partial t^{2}} = \frac{\partial}{\partial x}\left(gH\frac{\partial \eta}{\partial x}\right)$

where $\eta$ is the surface height perturbation [m], $H$ is the ocean depth [m], and $g$ is gravity [$m$ $s^{-2}$]. Using a scheme that is centered in both time and space, we discretize the wave equation as (see derivation in class):

$\frac{\eta_{j}^{(n+1)}-2\eta_{j}^{(n)}+\eta_{j}^{(n-1)}}{\Delta t^{2}} = \frac{g \bar{H}_{j}}{\Delta x^{2}} \left(\eta_{j+1}^{(n)}-2\eta_{j}^{(n)}+\eta_{j-1}^{(n)}\right) + g\frac{\bar{H}_{j+1}-\bar{H}_{j-1}}{2\Delta x} \frac{\eta_{j+1}^{(n)}-\eta_{j-1}^{(n)}}{2\Delta x}$

where $\bar{H}$ is the reference ocean depth [m].

In terms of numerics, there are important questions that you should think about before going into the code:
- What is neglected in our discretization of the shallow water wave equation?
- What initial conditions can we set?
- How do we solve for the first time step?

Finally, note the shallow water wave equation imnplies a wave speed propagation given by $v=\sqrt{gH}$.

In [ ]:
import os
import sys
import math
import copy
import time
import numpy as np
import matplotlib.pyplot as plt

Set up the x-grid and the bathymetry

In [ ]:
deltax  = 2000.0 #[m]
xgrid   = np.arange(-4e6,4e6+1,deltax) #[m]
xnum    = len(xgrid)
### Bathymetry with a continental slope ###
contslope_pos   = (3/4)*np.max(xgrid)
contslope_slope = 0.003
bathy           = -4000*np.ones(xnum)
inds            = xgrid>=contslope_pos
bathy[inds]     = bathy[inds]+contslope_slope*(xgrid[inds]-contslope_pos)
bathy_a         = abs(bathy)

Set the initial peturbation as a Gaussian bump

In [ ]:
### Gaussian bump ###
bump_h  = 10.0
bump_w  = 50000.0
bump_c  = 0
uu_0    = bump_h*np.exp((-1/2)*((xgrid-bump_c)**2)/(bump_w**2))

Define the parameters.

In [ ]:
grv       = 9.81 #[m s-2]
secperh   = 3600 #[s]
mperkm    = 1000 #[m]
### Time parameters ###
tmend_h    = 5 #[hour]
tmend_sec  = tmend_h*secperh #[s]
deltat_sec = 1 #[s]
nsteps     = int(tmend_sec/deltat_sec)
ttplot     = np.linspace(0.001*nsteps,nsteps-1,8).astype(int).tolist()

Initialize our solution array.

In [ ]:
uu_cur = copy.deepcopy(uu_0)

Solve with a scheme centered in time and centered in space.

In [ ]:
uu_old    = np.nan*np.ones(xnum)
uu_old_2  = np.nan*np.ones(xnum)
to_plot   = np.zeros((len(ttplot),xnum))
for tt in range(nsteps):
    # Copy old uu field #
    uu_old_2        = copy.deepcopy(uu_old)
    uu_old          = copy.deepcopy(uu_cur)
    # Centered spatial differences #
    sp_term1        = uu_old[2:]-2*uu_old[1:-1]+uu_old[0:-2]
    sp_term2        = ((bathy_a[2:]-bathy_a[0:-2])*(uu_old[2:]-uu_old[0:-2]))
    fac1            = ((np.sqrt(grv*bathy_a)*deltat_sec/deltax)**2)[1:-1]
    fac2            = (np.sqrt(grv)*deltat_sec/(2*deltax))**2
    if(tt==0):
        # Initial condition: du/dt=0 #
        uu_cur[1:-1] = uu_old[1:-1] + (1/2)*fac1*sp_term1 + fac2*sp_term2
    elif(tt>0):
        uu_cur[1:-1] = -1*uu_old_2[1:-1] + 2*uu_old[1:-1] + fac1*sp_term1 + fac2*sp_term2
    # Boundary conditions #
    uu_cur[0]  = copy.deepcopy(uu_cur[2])
    uu_cur[-1] = copy.deepcopy(uu_cur[-3])
    # Save for plotting #
    if(tt in ttplot):
        to_plot[ttplot.index(tt),:] = copy.deepcopy(uu_cur)

And make a figure.

In [ ]:
xgrid_km = xgrid/mperkm
plcols = plt.cm.jet
fig = plt.figure(figsize=(10,9))
ax = plt.subplot(211)
for kk in range(len(ttplot)):
    ax.plot(xgrid_km,to_plot[kk,:],c=plcols(kk/len(ttplot)),label=f'{np.round(ttplot[kk]*deltat_sec/secperh,1)} h')
ax.legend(loc='best',fontsize=12,ncol=2)
ax.tick_params(which='major',axis='both',labelsize=12)    
ax.set_xlim([-10*deltax/mperkm,np.max(xgrid_km)])
ax.set_xticks([])
ax.set_ylabel('Wave height [m]',fontsize=12)
ax = plt.subplot(212)
ax.plot(xgrid_km,bathy,c='k',linewidth=2.5)
ax.tick_params(which='major',axis='both',labelsize=12)    
ax.set_xlim([-10*deltax/mperkm,np.max(xgrid_km)])
ax.set_ylim([1.1*np.min(bathy),0])
ax.set_xlabel('x [km]',fontsize=12)
ax.set_ylabel('Bathymetry [m]',fontsize=12)
fig.tight_layout()    